# Preparing the launch file

In [1]:
import numpy as np
import pandas as pd


In [108]:
#pd.read_csv("./source-db/mp-160/POSCAR-unitcell")

pos_path = './source-db/mp-160/POSCAR-unitcell'
sample_path = './launch_thermal_si.py'
pos_file = open(pos_path,"r")

scaled_coordinates=open('./source-db/mp-160/scaled_coordinates.dat', "w")
linenumber=0
for line in pos_file.readlines():
    linenumber=linenumber+1
    if 3<=linenumber<=5:
        print("[",line,"],")


    if linenumber>=9:
        scaled_coordinates.write("("+line+"),\n")
    

scaled_coordinates.close()

[      4.8786175399999996    0.0000000000000000    0.0000000000000000
 ],
[     -2.4393087699999998    4.2250067299999996    0.0000000000000000
 ],
[      0.0000000000000000    0.0000000000000000   12.5104310900000009
 ],


# Parallel phono3py calculations using openMP

Togo-sensei found my phono3py in boston-lab did not use the parallel thread. I tried restalled phonopy and phono3py for several times, but didn' figure it out.

Today, after checking my .bashrc file line by line, I found the problem is caused my setting for the number of thread.

export OMP_NUM_THREADS = 1

After removing this line, and source .bashrc. The threads can be parallel.

## An example to get lattice thermal conductivity

be sure you have setup a virtual environment for aiida, and have a work job with its pk number (e.g. 186270 in my example)

In [ ]:
open verdi shell

In [ ]:
n3 = load_node(186270)

In [ ]:
n3.out.kappa

In [ ]:
<ArrayData: uuid: af240b4c-29ad-4c80-8691-a52a65d1587d (pk: 187185)>

In [ ]:
n3.out.kappa.get_arraynames()

In [ ]:
Out[4]:
[u'group_velocity',
 u'temperature',
 u'mesh',
 u'heat_capacity',
 u'frequency',
 u'gv_by_gv',
 u'gamma',
 u'weight',
 u'kappa',
 u'kappa_unit_conversion',
 u'qpoint',
 
 u'mode_kappa']

In [ ]:
f = n3.out.kappa

In [ ]:
f.get_array('kappa')[30] #this will print the ltc tensors

# output to json file example

In [3]:
import json
data = {
    'name': 'TiO2',
    'ID': 'mp-100',
    'ltc': [100,200]
}
json_str = json.dumps(data)

In [4]:
json_str

'{"name": "TiO2", "ID": "mp-100", "ltc": [100, 200]}'

In [ ]:
70325

<module 'json' from '/home/ywfang/miniconda3/envs/aiida/lib/python2.7/json/__init__.pyc'>

# A practical example

In [62]:
import json
try:
    from pymatgen.matproj.rest import MPRester #matproj.rest is moved to ext.matproj in new pymatgen versions
except ImportError: 
    from pymatgen.ext.matproj import MPRester
import os
data = {}
#structure_id is the ID of the structure in Materials Project


raster = MPRester(os.environ['PMG_MAPI_KEY'])
def basic_info_from_mp(structure_id):
    material_name = raster.query(structure_id,['pretty_formula'])
#    mg_structure = raster.get_structure_by_material_id(structure_id)
#    material_name = mg_structure.formula.replace('1', '').replace(' ', '')
    return(structure_id, material_name[0]['pretty_formula'])

data['compound'] = []
data['compound'].append({
    'structure_id': 'mp-149',
    'material_name': basic_info_from_mp('mp-149')[1]
})

with open('data.json', 'w') as outfile:
    json.dump(data, outfile)

In [63]:
%cat data.json

{"compound": [{"structure_id": "mp-149", "material_name": "Si"}]}

# load the requested data to thermaldata.json

In [60]:
import json
   
try:
    from pymatgen.matproj.rest import MPRester #matproj.rest is moved to ext.matproj in new pymatgen versions
except ImportError: 
    from pymatgen.ext.matproj import MPRester

import os
import pandas as pd

data = {}
#####################dict#####################################
#structure_id is the ID of the structure in Materials Project#
#ltc_cutoff (cutoff= 4, 5, 6... angstrom)                    #
#ltc (lattice thermal conductivity) unit: W/(m.k) at 300 K.  #
####################I/O file##################################
#finished_jobs.txt: it is a file contains the pk_number in my aiida calculations
#thermaldata.json: json files contaisn the lattice thermal conductivity data at 300 K

raster = MPRester(os.environ['PMG_MAPI_KEY'])
#PMG_MAPI_KEY should be set in .bashrc file or using command "export PMG_MAPI_KE = 'your key'"
def basic_info_from_mp(structure_id):
    material_name = raster.query(structure_id,['pretty_formula'])
    return(structure_id, material_name[0]['pretty_formula'])

job_df = pd.read_csv('./finished_jobs.txt')
structure_id_series = job_df['structure_id']
data['compound'] = []
index_in_job = 0
for i in job_df['structure_id'].tolist():
    data['compound'].append({
        'structure_id': i,
        'material_name': basic_info_from_mp(i)[1],
        'ltc4': job_df.iloc[index_in_job, 2],
        'ltc5': job_df.iloc[index_in_job, 3],
        'ltc6': job_df.iloc[index_in_job, 4],
        'ltc7': job_df.iloc[index_in_job, 5],
        'ltc8': job_df.iloc[index_in_job, 6],
        'ltc9': job_df.iloc[index_in_job, 7]
    })
    index_in_job = int(index_in_job) + 1

with open('thermaldata.json', 'w') as outfile:
    json.dump(data, outfile)

In [ ]:
job_df 

In [66]:
%more thermaldata.json

# LTC as a function of cutoff for GaN

In [4]:
#read thermaldata.json and convert it to a DataFrame
import pandas as pd
from pandas import DataFrame
import json
with open('thermaldata.json', 'r') as outfile:
    newdata=json.load(outfile)['compound']
df=DataFrame(newdata)
df.head(1)

,ltc4,ltc5,ltc6,ltc7,ltc8,ltc9,material_name,structure_id
0,81.860504,79.058721,NaN,NaN,NaN,NaN,Si,mp-149


In [35]:
job_df.iloc[0,2]

81.860504400000011

# Convergence criteria in lattice thermal conductivity calculations

In our code, criteria is achived by using "numpy.allclose" function. 
The usage of "numpy.allclose": numpy.allclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False). Here, a and b are numpy arrarys. The relative difference (rtol * abs(b)) and the absolute difference atol are added together to compare against the absolute difference between a and b.

Specifically in our thermal conductivity calculations, a and b are replaced with current kappa numpy.ndarray and previous kappa numpy.ndarray, respectively. rtol and atol are chosen to be 0.3 and 0.1 respectively. 

code segment:

is_converged = np.allclose(self.ctx.conductivity, self.ctx.prev_conductivity,
                                           rtol=float(self.inputs.rtol),
                                           atol=float(self.inputs.atol))

## References:

[1. Using iloc, loc, & ix to select rows and columns in Pandas DataFrames](https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/)

[2. Phonon database at 2017-06-21](http://phonondb.mtl.kyoto-u.ac.jp/ph20170621/index.html)

[3. numpy.allclose at SciPy.org](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.allclose.html)

[4. Reading and Writing JSON to a File in Python](http://stackabuse.com/reading-and-writing-json-to-a-file-in-python/)